# Model

In [1]:
import torch
import torch.nn as nn

class FirstFeature(nn.Module):
    """
    - Mục đích: Tạo feature map ban đầu từ input
    - Thành phần: Một lớp convolution duy nhất theo sau là hàm LeakyReLU. convolution này
    sử dụng kích thước kernel là 1, bước nhảy là 1 và không padding. Đây là một lớp đơn giản
    được thiết kế để mở rộng số lượng channel cho feature map
    """
    def __init__(self, in_channels, out_channels):
        super(FirstFeature, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias=False)
        )
        
    def forward(self, x):
        return self.conv(x)

class ConvBlock(nn.Module):
    """
    - Mục đích: Khối convolution cơ bản để trích xuất đặc trưng
    - Thành phần: Hai nhóm Conv-BatchNorm-LeakyReLU liên tục. Khối này là một khối cơ
    bản trong U-Net, được sử dụng cho cả down-sampling và up-sampling
    """
    def __init__(self, in_channels, out_channels):
        super(ConvBlock, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(inplace=True),
        )
        
    def forward(self, x):
        return self.conv(x)
    
class Encoder(nn.Module):
    """
    - Mục đích: Để giảm size của feature map và trích xuất các high-level feature
    - Thành phần: Một lớp Max Pooling theo sau là ConvBlock. Max Pooling giảm kích thước
    xuống một nửa, trong khi ConvBlock xử lý các đặc trưng.
    """
    def __init__(self, in_channels, out_channels) -> None:
        super().__init__()
        self.encoder = nn.Sequential(
            nn.MaxPool2d(2),
            ConvBlock(in_channels, out_channels)
        )

    def forward(self, x):
        x = self.encoder(x)
        return x


class Decoder(nn.Module):
    """
    - Mục đích: Để tăng kích thước feature map và kết hợp với feature map tương ứng từ Encoder
    (skip connection).
    - Thành phần: Upsampling (sử dụng nội suy bilinear) để tăng kích thước không gian. Một
    lớp convolution để giảm số lượng channel. Một ConvBlock để xử lý các feature được ghép
    (từ lớp upsampling và skip connection).
    """
    def __init__(self, in_channels, out_channels):
        super(Decoder, self).__init__()
        self.conv = nn.Sequential(
            nn.UpsamplingBilinear2d(scale_factor=2),
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(),
        )
        self.conv_block = ConvBlock(in_channels, out_channels)

    def forward(self, x, skip):
        x = self.conv(x)
        x = torch.concat([x, skip], dim=1)
        x = self.conv_block(x)
        return x


class FinalOutput(nn.Module):
    """
    - Mục đích: Tạo ra đầu ra cuối cùng từ feature map cuối cùng
    - Thành phần:  Một lớp convolution với hàm Tanh. Điều này giảm số lượng channel đầu ra
    xuống bằn số lượng channel của ảnh màu
    """
    def __init__(self, in_channels, out_channels):
        super(FinalOutput, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 1, 1, 0, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        return self.conv(x)


class Unet(nn.Module):
    """
    - Mục đích: Kết hợp tất cả các thành phần trên thành một kiến trúc U-Net đầy đủ.
    - Thành phần:  Xử lý ảnh đầu vào bằng FirstFeature và ConvBlock. Bốn lớp Encoder với số
    channel tăng dần, mỗi lớp tiếp tục downsample và xử lý feature map. Bốn lớp Decoder với số
    channel giảm dần, mỗi lớp tăng kích thước, kết hợp đặc trưng từ encoder (skip connection).
    Một lớp FinalOutput để tạo ra ảnh đã được xử lý.
    - Forward:  Đầu vào được xử lý qua các convolution ban đầu. Sau đó được downsample 4 lần,
    rồi được up sample lên 4 lần mỗi lần kết hợp với feature từ encoder. Cuối cùng đi qua lớp
    convolution cuối cùng để tạp ảnh đã xử lý
    """
    def __init__(
            self, n_channels=3, n_classes=3, features=[64, 128, 256, 512],
    ):
        super(Unet, self).__init__()

        self.n_channels = n_channels
        self.n_classes = n_classes

        self.in_conv1 = FirstFeature(n_channels, 64)
        self.in_conv2 = ConvBlock(64, 64)

        self.enc_1 = Encoder(64, 128)
        self.enc_2 = Encoder(128, 256)
        self.enc_3 = Encoder(256, 512)
        self.enc_4 = Encoder(512, 1024)

        self.dec_1 = Decoder(1024, 512)
        self.dec_2 = Decoder(512, 256)
        self.dec_3 = Decoder(256, 128)
        self.dec_4 = Decoder(128, 64)

        self.out_conv = FinalOutput(64, n_classes)


    def forward(self, x):
        x = self.in_conv1(x)
        x1 = self.in_conv2(x)
        
        x2 = self.enc_1(x1)
        x3 = self.enc_2(x2)
        x4 = self.enc_3(x3)
        x5 = self.enc_4(x4)
        
        x = self.dec_1(x5, x4)
        x = self.dec_2(x, x3)
        x = self.dec_3(x, x2)
        x = self.dec_4(x, x1)
        
        x = self.out_conv(x)
        
        return x
    